# Basic statistics and visualization technqiues

Our goal in this notebook is to introduce some basic notions from statistics, along with some visualization techniques.  As usual, let's start by importing the necessary libraries.

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns